In [94]:
import pandas as pd
from sklearn.pipeline import Pipeline, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer

In [52]:
# leemos el dataset limpio de partida, con las columas con las que vamos a trabajar

df_students_clean = pd.read_csv('C:\\Users\\saraf\\OneDrive\\Documentos\\Data Science and AI\\capstone project\\data\\df_students_clean.csv')

df_students_clean.head()

,ST267Q01JA,ST267Q02JA,ST267Q03JA,ST267Q04JA,ST267Q05JA,ST267Q06JA,ST267Q07JA,ST267Q08JA,ST270Q01JA,ST270Q02JA,...,PV1SCIE,PV2SCIE,PV3SCIE,PV4SCIE,PV5SCIE,PV6SCIE,PV7SCIE,PV8SCIE,PV9SCIE,PV10SCIE
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,506.912,520.943,445.915,447.578,510.355,469.425,485.229,520.293,509.465,520.461
1,Agree,Disagree,NaN,NaN,Agree,Strongly agree,Agree,NaN,Some lessons,Some lessons,...,472.056,497.419,583.550,486.117,543.515,501.654,522.876,518.897,540.951,555.862
2,NaN,Agree,Agree,NaN,Agree,Agree,NaN,Disagree,Some lessons,Some lessons,...,336.125,383.288,390.317,360.386,389.356,424.082,377.563,422.321,363.447,431.934
3,Agree,Disagree,Agree,Strongly disagree,NaN,NaN,Agree,NaN,Every lesson,Every lesson,...,413.822,324.202,343.227,284.083,330.103,351.840,353.562,381.369,366.970,350.518
4,NaN,Strongly disagree,Strongly disagree,NaN,Strongly disagree,Strongly disagree,NaN,Agree,Most lessons,Most lessons,...,530.906,472.011,492.387,431.729,524.883,420.419,529.333,559.941,496.913,488.453


In [53]:
df_students_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30800 entries, 0 to 30799
Data columns (total 88 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ST267Q01JA  18516 non-null  object 
 1   ST267Q02JA  18403 non-null  object 
 2   ST267Q03JA  18464 non-null  object 
 3   ST267Q04JA  18378 non-null  object 
 4   ST267Q05JA  18545 non-null  object 
 5   ST267Q06JA  18283 non-null  object 
 6   ST267Q07JA  18381 non-null  object 
 7   ST267Q08JA  18550 non-null  object 
 8   ST270Q01JA  28806 non-null  object 
 9   ST270Q02JA  28746 non-null  object 
 10  ST270Q03JA  28756 non-null  object 
 11  ST270Q04JA  28769 non-null  object 
 12  ST297Q01JA  29406 non-null  object 
 13  ST300Q01JA  13422 non-null  object 
 14  ST300Q02JA  13490 non-null  object 
 15  ST300Q03JA  13511 non-null  object 
 16  ST300Q04JA  13298 non-null  object 
 17  ST300Q05JA  13332 non-null  object 
 18  ST300Q06JA  13293 non-null  object 
 19  ST300Q07JA  13398 non-nul

In [54]:
def obtener_significado(codigo, df_compendio):
    resultado = df_compendio.loc[df_compendio['Sigla'] == codigo, 'Explicacion']
    return resultado.iloc[0] if not resultado.empty else "Not found"

In [55]:
compendio = pd.read_excel('compendio_df_students.xlsx')

Imprimimos las columnas con las que estamos trabajando

In [56]:
for column in df_students_clean.columns:
    print(f"{column}: {obtener_significado(column, compendio)}")

ST267Q01JA: Agree/disagree: The teachers at my school are respectful towards me.
ST267Q02JA: Agree/disagree: If I walked into my classes upset, my teachers would be concerned about me.
ST267Q03JA: Agree/disagree: If I came back to visit my school 3 years from now, my teachers would be excited to see me.
ST267Q04JA: Agree/disagree: I feel intimidated by the teachers at my school.
ST267Q05JA: Agree/disagree: When my teachers ask how I am doing, they are really interested in my answer.
ST267Q06JA: Agree/disagree: The teachers at my school are friendly towards me.
ST267Q07JA: Agree/disagree: The teachers at my school are interested in students' well-being.
ST267Q08JA: Agree/disagree: The teachers at my school are mean towards me.
ST270Q01JA: How often: The teacher shows an interest in every student's learning.
ST270Q02JA: How often: The teacher gives extra help when students need it.
ST270Q03JA: How often: The teacher helps students with their learning.
ST270Q04JA: How often: The teacher c

In [57]:
df_students_clean.head()

,ST267Q01JA,ST267Q02JA,ST267Q03JA,ST267Q04JA,ST267Q05JA,ST267Q06JA,ST267Q07JA,ST267Q08JA,ST270Q01JA,ST270Q02JA,...,PV1SCIE,PV2SCIE,PV3SCIE,PV4SCIE,PV5SCIE,PV6SCIE,PV7SCIE,PV8SCIE,PV9SCIE,PV10SCIE
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,506.912,520.943,445.915,447.578,510.355,469.425,485.229,520.293,509.465,520.461
1,Agree,Disagree,NaN,NaN,Agree,Strongly agree,Agree,NaN,Some lessons,Some lessons,...,472.056,497.419,583.550,486.117,543.515,501.654,522.876,518.897,540.951,555.862
2,NaN,Agree,Agree,NaN,Agree,Agree,NaN,Disagree,Some lessons,Some lessons,...,336.125,383.288,390.317,360.386,389.356,424.082,377.563,422.321,363.447,431.934
3,Agree,Disagree,Agree,Strongly disagree,NaN,NaN,Agree,NaN,Every lesson,Every lesson,...,413.822,324.202,343.227,284.083,330.103,351.840,353.562,381.369,366.970,350.518
4,NaN,Strongly disagree,Strongly disagree,NaN,Strongly disagree,Strongly disagree,NaN,Agree,Most lessons,Most lessons,...,530.906,472.011,492.387,431.729,524.883,420.419,529.333,559.941,496.913,488.453


## Preprocesado inicial

Descartamos columnas vacías

In [58]:
# descartar columnas que empiezan por PA

df_students_clean = df_students_clean.drop(df_students_clean.filter(regex='^PA').columns, axis=1)



In [59]:
df_students_clean.shape

(30800, 71)

In [60]:
df_students_clean.columns

Index(['ST267Q01JA', 'ST267Q02JA', 'ST267Q03JA', 'ST267Q04JA', 'ST267Q05JA',
       'ST267Q06JA', 'ST267Q07JA', 'ST267Q08JA', 'ST270Q01JA', 'ST270Q02JA',
       'ST270Q03JA', 'ST270Q04JA', 'ST297Q01JA', 'ST300Q01JA', 'ST300Q02JA',
       'ST300Q03JA', 'ST300Q04JA', 'ST300Q05JA', 'ST300Q06JA', 'ST300Q07JA',
       'ST300Q08JA', 'ST300Q09JA', 'ST300Q10JA', 'ST348Q04JA', 'ST348Q08JA',
       'ST353Q01JA', 'ST353Q02JA', 'ST353Q05JA', 'ST353Q06JA', 'ST353Q07JA',
       'ST353Q08JA', 'WB155Q08HA', 'WB155Q09HA', 'WB163Q01HA', 'WB163Q02HA',
       'WB163Q03HA', 'WB163Q04HA', 'WB163Q05HA', 'WB163Q06HA', 'WB163Q07HA',
       'WB163Q08HA', 'PV1MATH', 'PV2MATH', 'PV3MATH', 'PV4MATH', 'PV5MATH',
       'PV6MATH', 'PV7MATH', 'PV8MATH', 'PV9MATH', 'PV10MATH', 'PV1READ',
       'PV2READ', 'PV3READ', 'PV4READ', 'PV5READ', 'PV6READ', 'PV7READ',
       'PV8READ', 'PV9READ', 'PV10READ', 'PV1SCIE', 'PV2SCIE', 'PV3SCIE',
       'PV4SCIE', 'PV5SCIE', 'PV6SCIE', 'PV7SCIE', 'PV8SCIE', 'PV9SCIE',
       'PV10SC

Hacemos la media para cada uno de los tipos de valores plausibles (MATH, READ, SCIE)

In [61]:
# crear una nueva columna que sea la media de las columnas que contienen MATH
df_students_clean['MATH_mean'] = df_students_clean.filter(like='MATH').mean(axis=1)

# crear una nueva columna que sea la media de las columnas que contienen READ
df_students_clean['READ_mean'] = df_students_clean.filter(like='READ').mean(axis=1)

# crear una nueva columna que sea la media de las columnas que contienen SCIE
df_students_clean['SCIE_mean'] = df_students_clean.filter(like='SCIE').mean(axis=1)

In [62]:
df_students_clean.head(4)

,ST267Q01JA,ST267Q02JA,ST267Q03JA,ST267Q04JA,ST267Q05JA,ST267Q06JA,ST267Q07JA,ST267Q08JA,ST270Q01JA,ST270Q02JA,...,PV4SCIE,PV5SCIE,PV6SCIE,PV7SCIE,PV8SCIE,PV9SCIE,PV10SCIE,MATH_mean,READ_mean,SCIE_mean
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,447.578,510.355,469.425,485.229,520.293,509.465,520.461,471.9383,455.4310,493.6576
1,Agree,Disagree,NaN,NaN,Agree,Strongly agree,Agree,NaN,Some lessons,Some lessons,...,486.117,543.515,501.654,522.876,518.897,540.951,555.862,494.8669,537.5307,522.2897
2,NaN,Agree,Agree,NaN,Agree,Agree,NaN,Disagree,Some lessons,Some lessons,...,360.386,389.356,424.082,377.563,422.321,363.447,431.934,317.8654,348.4248,387.8819
3,Agree,Disagree,Agree,Strongly disagree,NaN,NaN,Agree,NaN,Every lesson,Every lesson,...,284.083,330.103,351.840,353.562,381.369,366.970,350.518,372.5356,424.7139,349.9696


In [63]:
df_students_clean[['MATH_mean', 'READ_mean', 'SCIE_mean']].head()

,MATH_mean,READ_mean,SCIE_mean
0,471.9383,455.4310,493.6576
1,494.8669,537.5307,522.2897
2,317.8654,348.4248,387.8819
3,372.5356,424.7139,349.9696
4,485.6010,497.4628,494.6975


Eliminamos el resto de valores plausibles, que no nos van a hacer falta

In [64]:
# eliminar columnas que empiezan por PV

df_students_clean = df_students_clean.drop(df_students_clean.filter(regex='^PV').columns, axis=1)

In [65]:
df_students_clean.columns

Index(['ST267Q01JA', 'ST267Q02JA', 'ST267Q03JA', 'ST267Q04JA', 'ST267Q05JA',
       'ST267Q06JA', 'ST267Q07JA', 'ST267Q08JA', 'ST270Q01JA', 'ST270Q02JA',
       'ST270Q03JA', 'ST270Q04JA', 'ST297Q01JA', 'ST300Q01JA', 'ST300Q02JA',
       'ST300Q03JA', 'ST300Q04JA', 'ST300Q05JA', 'ST300Q06JA', 'ST300Q07JA',
       'ST300Q08JA', 'ST300Q09JA', 'ST300Q10JA', 'ST348Q04JA', 'ST348Q08JA',
       'ST353Q01JA', 'ST353Q02JA', 'ST353Q05JA', 'ST353Q06JA', 'ST353Q07JA',
       'ST353Q08JA', 'WB155Q08HA', 'WB155Q09HA', 'WB163Q01HA', 'WB163Q02HA',
       'WB163Q03HA', 'WB163Q04HA', 'WB163Q05HA', 'WB163Q06HA', 'WB163Q07HA',
       'WB163Q08HA', 'MATH_mean', 'READ_mean', 'SCIE_mean'],
      dtype='object')

### Transformación de las columnas a numéricas

Convertimos las columnas a numéricas.

In [66]:
# creamos los mappings que necesitemos

mapping_agree =  {"Strongly disagree": 0, "Disagree": 1, "Agree": 2, "Strongly agree": 3}
mapping_lesson = {"Never or almost never": 0, "Some lessons": 1, "Most lessons": 2, "Every lesson": 3}
mapping_checked = {"Not Checked": 0, "Checked": 1}
mapping_freq = {"Never or almost never": 0, "About once or twice a year": 1, "About once or twice a month": 2, "About once a week": 3, "Every day or almost every day": 4}
mapping_a_few = {'Never': 0, 'A few times': 1, 'About once or twice a week': 2, 'Every day or almost every day': 3 }
mapping_satisfied = {"Totally satisfied": 3 ,'Satisfied': 2, "Not satisfied": 1 , "Not at all satisfied": 0}
mapping_almost = {"Almost never": 0, "Sometimes": 1, "Almost always": 2}

In [67]:
# create transformers from mappings
def map_values(x, mapping):
    if isinstance(x, pd.DataFrame):
        # If x is a DataFrame, apply the mapping to each column
        return x.apply(lambda col: col.map(mapping))
    else:
        # If x is a Series, apply the mapping directly
        return x.map(mapping)

In [68]:
transformer_agree = FunctionTransformer(lambda x: map_values(x, mapping_agree))
transformer_lesson = FunctionTransformer(lambda x: map_values(x, mapping_lesson))
transformer_checked = FunctionTransformer(lambda x: map_values(x, mapping_checked))
transformer_freq = FunctionTransformer(lambda x: map_values(x, mapping_freq))
transformer_a_few = FunctionTransformer(lambda x: map_values(x, mapping_a_few))
transformer_satisfied = FunctionTransformer(lambda x: map_values(x, mapping_satisfied))
transformer_almost = FunctionTransformer(lambda x: map_values(x, mapping_almost))

In [69]:
columns_agree = ["ST267Q01JA", "ST267Q02JA", "ST267Q03JA", "ST267Q04JA", "ST267Q05JA", "ST267Q06JA", "ST267Q07JA", "ST267Q08JA"]
columns_lesson = ["ST270Q01JA", "ST270Q02JA", "ST270Q03JA", "ST270Q04JA"]
columns_checked = ["ST297Q01JA"]
columns_freq = ["ST300Q01JA", "ST300Q02JA", "ST300Q03JA", "ST300Q04JA", "ST300Q05JA", "ST300Q06JA", "ST300Q07JA", "ST300Q08JA",
                 "ST300Q09JA", "ST300Q10JA"]
columns_a_few = ['ST348Q04JA', 'ST348Q08JA', "ST353Q01JA", "ST353Q02JA", "ST353Q05JA", "ST353Q06JA", "ST353Q07JA", "ST353Q08JA"]
columns_satisfied = ['WB155Q08HA', 'WB155Q09HA']
columns_almost = ["WB163Q01HA", "WB163Q02HA", "WB163Q03HA", "WB163Q04HA", "WB163Q05HA", "WB163Q06HA", "WB163Q07HA", "WB163Q08HA"]

In [70]:
transform_pipe = ColumnTransformer(
    transformers = [
        ("transformer_agree", transformer_agree, columns_agree),
        ("transformer_lesson", transformer_lesson, columns_lesson),
        ("transformer_checked", transformer_checked, columns_checked),
        ("transformer_freq", transformer_freq, columns_freq),
        ("transformer_a_few", transformer_a_few, columns_a_few),
        ("transformer_satisfied", transformer_satisfied, columns_satisfied),
        ("transformer_almost", transformer_almost, columns_almost)
    ],
    remainder="passthrough"
)

In [71]:
df_students_num = transform_pipe.fit_transform(df_students_clean)

# transformamos el resultado a un dataframe
df_students_num = pd.DataFrame(df_students_num)

# Get all column names from the original dataframe
all_columns = (columns_agree + columns_lesson + columns_checked + columns_freq + 
               columns_a_few + columns_satisfied + columns_almost)

# Get the remaining columns (those not transformed)
remaining_columns = [col for col in df_students_clean.columns if col not in all_columns]

# Combine all column names in the order they appear in the transformed data
column_names = all_columns + remaining_columns

In [72]:
# Assign column names to the transformed dataframe
df_students_num.columns = column_names

df_students_num.head(6)

,ST267Q01JA,ST267Q02JA,ST267Q03JA,ST267Q04JA,ST267Q05JA,ST267Q06JA,ST267Q07JA,ST267Q08JA,ST270Q01JA,ST270Q02JA,...,WB163Q02HA,WB163Q03HA,WB163Q04HA,WB163Q05HA,WB163Q06HA,WB163Q07HA,WB163Q08HA,MATH_mean,READ_mean,SCIE_mean
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,471.9383,455.4310,493.6576
1,2.0,1.0,NaN,NaN,2.0,3.0,2.0,NaN,1.0,1.0,...,2.0,2.0,2.0,2.0,0.0,0.0,2.0,494.8669,537.5307,522.2897
2,NaN,2.0,2.0,NaN,2.0,2.0,NaN,1.0,1.0,1.0,...,1.0,2.0,2.0,1.0,1.0,0.0,2.0,317.8654,348.4248,387.8819
3,2.0,1.0,2.0,0.0,NaN,NaN,2.0,NaN,3.0,3.0,...,1.0,2.0,2.0,2.0,0.0,1.0,2.0,372.5356,424.7139,349.9696
4,NaN,0.0,0.0,NaN,0.0,0.0,NaN,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,0.0,0.0,2.0,485.6010,497.4628,494.6975
5,3.0,3.0,NaN,0.0,NaN,3.0,3.0,NaN,3.0,3.0,...,2.0,2.0,2.0,2.0,1.0,0.0,2.0,432.8490,323.1210,395.8614


In [73]:
# comparamos con el dataframe original
df_students_clean.head(6)

,ST267Q01JA,ST267Q02JA,ST267Q03JA,ST267Q04JA,ST267Q05JA,ST267Q06JA,ST267Q07JA,ST267Q08JA,ST270Q01JA,ST270Q02JA,...,WB163Q02HA,WB163Q03HA,WB163Q04HA,WB163Q05HA,WB163Q06HA,WB163Q07HA,WB163Q08HA,MATH_mean,READ_mean,SCIE_mean
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,471.9383,455.4310,493.6576
1,Agree,Disagree,NaN,NaN,Agree,Strongly agree,Agree,NaN,Some lessons,Some lessons,...,Almost always,Almost always,Almost always,Almost always,Almost never,Almost never,Almost always,494.8669,537.5307,522.2897
2,NaN,Agree,Agree,NaN,Agree,Agree,NaN,Disagree,Some lessons,Some lessons,...,Sometimes,Almost always,Almost always,Sometimes,Sometimes,Almost never,Almost always,317.8654,348.4248,387.8819
3,Agree,Disagree,Agree,Strongly disagree,NaN,NaN,Agree,NaN,Every lesson,Every lesson,...,Sometimes,Almost always,Almost always,Almost always,Almost never,Sometimes,Almost always,372.5356,424.7139,349.9696
4,NaN,Strongly disagree,Strongly disagree,NaN,Strongly disagree,Strongly disagree,NaN,Agree,Most lessons,Most lessons,...,Almost always,Almost always,Almost always,Almost always,Almost never,Almost never,Almost always,485.6010,497.4628,494.6975
5,Strongly agree,Strongly agree,NaN,Strongly disagree,NaN,Strongly agree,Strongly agree,NaN,Every lesson,Every lesson,...,Almost always,Almost always,Almost always,Almost always,Sometimes,Almost never,Almost always,432.8490,323.1210,395.8614


### Añadimos columnas necesarias

In [74]:
# vamos a obtener student_id por si lo necesitamos

df_students_spain = pd.read_csv('C:\\Users\\saraf\\OneDrive\\Documentos\\Data Science and AI\\capstone project\\data\\df_students_spain.csv')

In [75]:
# cogemos columna student_id
student_id = df_students_spain['CNTSTUID']

# eliminamos df_students_spain
del df_students_spain

# añadimos student_id al dataframe con el que estamos trabajando
df_students_num['student_id'] = student_id



In [76]:
df_students_num.head()

,ST267Q01JA,ST267Q02JA,ST267Q03JA,ST267Q04JA,ST267Q05JA,ST267Q06JA,ST267Q07JA,ST267Q08JA,ST270Q01JA,ST270Q02JA,...,WB163Q03HA,WB163Q04HA,WB163Q05HA,WB163Q06HA,WB163Q07HA,WB163Q08HA,MATH_mean,READ_mean,SCIE_mean,student_id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,471.9383,455.4310,493.6576,72400001.0
1,2.0,1.0,NaN,NaN,2.0,3.0,2.0,NaN,1.0,1.0,...,2.0,2.0,2.0,0.0,0.0,2.0,494.8669,537.5307,522.2897,72400003.0
2,NaN,2.0,2.0,NaN,2.0,2.0,NaN,1.0,1.0,1.0,...,2.0,2.0,1.0,1.0,0.0,2.0,317.8654,348.4248,387.8819,72400006.0
3,2.0,1.0,2.0,0.0,NaN,NaN,2.0,NaN,3.0,3.0,...,2.0,2.0,2.0,0.0,1.0,2.0,372.5356,424.7139,349.9696,72400007.0
4,NaN,0.0,0.0,NaN,0.0,0.0,NaN,2.0,2.0,2.0,...,2.0,2.0,2.0,0.0,0.0,2.0,485.6010,497.4628,494.6975,72400010.0


### Creamos las variables que vamos a emplear para el clústering

La variables estarán construidas como una combinación de las columnas del dataset usado para la clasificación. A priori:

teach_friend (teacher friendliness): ST267Q01JA, ST267Q06JA

teach_mean (teacher meanness): ST267Q08JA

emo_impl_teach (emotional implication of teachers): ST267Q02JA, ST267Q03JA, ST267Q05JA, ST267Q07JA

teach_intim (teacher intimidation): ST267Q04JA

edu_impl_teach (educational implication of teachers): ST270Q01JA, ST270Q02JA, ST270Q03JA, ST270Q04JA

edu_impl_par (educational implication of parents): ST300Q01JA, ST300Q04JA, ST300Q07JA, ST300Q08JA, ST300Q09JA, ST348Q04JA

par_relation_sat (relationship with parents satisfaction): WB155Q08HA

teach_relation_sat (relationship with teachers satisfaction): WB155Q09HA

emo_imp_parents (emotional implication of parents): WB163Q01HA, WB163Q03HA, WB163Q04HA, WB163Q08HA

par_freedom (freedom granted by parents): WB163Q02HA, WB163Q05HA, WB163Q06HA, WB163Q07HA

In [77]:
df_students_num[['ST267Q01JA', 'ST267Q06JA']].head(20)

,ST267Q01JA,ST267Q06JA
0,NaN,NaN
1,2.0,3.0
2,NaN,2.0
3,2.0,NaN
4,NaN,0.0
5,3.0,3.0
6,NaN,2.0
7,3.0,2.0
8,2.0,2.0
9,3.0,NaN


Dado que tenemos muchos valores nulos, vamos a definir las nuevas categorías como la media de las columnas que involucran, de forma que solo contribuyan al resultado los valores no nulos.

Antes de realizar la transformación, vamos a invertir la puntuación de las columnas siguientes columnas, en las que una puntuación más alta sería considerada negativa:

WB163Q06HA, Try to control everything I do, almost never - sometimes...

WB163Q07HA, Treat me like a baby, almost never - ...

In [78]:
# Verify the changes
print("Example of unreversed values:")
print(df_students_num[['WB163Q06HA', 'WB163Q07HA']].head(5))

Example of unreversed values:
   WB163Q06HA  WB163Q07HA
0         NaN         NaN
1         0.0         0.0
2         1.0         0.0
3         0.0         1.0
4         0.0         0.0


In [79]:
# mapear 0 a 2, 1 a 1 y 2 a 0 para las columnas WB163Q06HA y WB163Q07HA

# Create reverse mapping for WB163Q06HA and WB163Q07HA
reverse_mapping = {0: 2, 1: 1, 2: 0}

# Apply the reverse mapping to the specific columns
df_students_num['WB163Q06HA'] = df_students_num['WB163Q06HA'].map(reverse_mapping)
df_students_num['WB163Q07HA'] = df_students_num['WB163Q07HA'].map(reverse_mapping)

# Verify the changes
print("Example of reversed values:")
print(df_students_num[['WB163Q06HA', 'WB163Q07HA']].head(5))


Example of reversed values:
   WB163Q06HA  WB163Q07HA
0         NaN         NaN
1         2.0         2.0
2         1.0         2.0
3         2.0         1.0
4         2.0         2.0


In [80]:
# create empty df

df_students_feat = pd.DataFrame()

In [81]:
# Create all combined columns
df_students_feat['teach_friend'] = df_students_num[['ST267Q01JA', 'ST267Q06JA']].mean(axis=1)
df_students_feat['teach_mean'] = df_students_num['ST267Q08JA']
df_students_feat['emo_impl_teach'] = df_students_num[['ST267Q02JA', 'ST267Q03JA', 'ST267Q05JA', 'ST267Q07JA']].mean(axis=1)
df_students_feat['teach_intim'] = df_students_num['ST267Q04JA']
df_students_feat['edu_impl_teach'] = df_students_num[['ST270Q01JA', 'ST270Q02JA', 'ST270Q03JA', 'ST270Q04JA']].mean(axis=1)
df_students_feat['edu_impl_par'] = df_students_num[['ST300Q01JA', 'ST300Q04JA', 'ST300Q07JA', 'ST300Q08JA', 'ST300Q09JA', 'ST348Q04JA']].mean(axis=1)
df_students_feat['par_relation_sat'] = df_students_num['WB155Q08HA']
df_students_feat['teach_relation_sat'] = df_students_num['WB155Q09HA']
df_students_feat['emo_imp_parents'] = df_students_num[['WB163Q01HA', 'WB163Q03HA', 'WB163Q04HA', 'WB163Q08HA']].mean(axis=1)
df_students_feat['par_freedom'] = df_students_num[['WB163Q02HA', 'WB163Q05HA', 'WB163Q06HA', 'WB163Q07HA']].mean(axis=1)

In [82]:
df_students_num.columns

Index(['ST267Q01JA', 'ST267Q02JA', 'ST267Q03JA', 'ST267Q04JA', 'ST267Q05JA',
       'ST267Q06JA', 'ST267Q07JA', 'ST267Q08JA', 'ST270Q01JA', 'ST270Q02JA',
       'ST270Q03JA', 'ST270Q04JA', 'ST297Q01JA', 'ST300Q01JA', 'ST300Q02JA',
       'ST300Q03JA', 'ST300Q04JA', 'ST300Q05JA', 'ST300Q06JA', 'ST300Q07JA',
       'ST300Q08JA', 'ST300Q09JA', 'ST300Q10JA', 'ST348Q04JA', 'ST348Q08JA',
       'ST353Q01JA', 'ST353Q02JA', 'ST353Q05JA', 'ST353Q06JA', 'ST353Q07JA',
       'ST353Q08JA', 'WB155Q08HA', 'WB155Q09HA', 'WB163Q01HA', 'WB163Q02HA',
       'WB163Q03HA', 'WB163Q04HA', 'WB163Q05HA', 'WB163Q06HA', 'WB163Q07HA',
       'WB163Q08HA', 'MATH_mean', 'READ_mean', 'SCIE_mean', 'student_id'],
      dtype='object')

Añadimos el resto de columnas necesarias

In [83]:
df_students_feat['MATH_mean'] = df_students_num['MATH_mean']
df_students_feat['READ_mean'] = df_students_num['READ_mean']
df_students_feat['SCIE_mean'] = df_students_num['SCIE_mean']
df_students_feat['student_id'] = df_students_num['student_id']

In [84]:
df_students_feat.columns

Index(['teach_friend', 'teach_mean', 'emo_impl_teach', 'teach_intim',
       'edu_impl_teach', 'edu_impl_par', 'par_relation_sat',
       'teach_relation_sat', 'emo_imp_parents', 'par_freedom', 'MATH_mean',
       'READ_mean', 'SCIE_mean', 'student_id'],
      dtype='object')

In [85]:
df_students_feat.head(10)

,teach_friend,teach_mean,emo_impl_teach,teach_intim,edu_impl_teach,edu_impl_par,par_relation_sat,teach_relation_sat,emo_imp_parents,par_freedom,MATH_mean,READ_mean,SCIE_mean,student_id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,471.9383,455.4310,493.6576,72400001.0
1,2.5,NaN,1.666667,NaN,1.25,NaN,3.0,2.0,2.00,2.00,494.8669,537.5307,522.2897,72400003.0
2,2.0,1.0,2.000000,NaN,1.00,4.00,2.0,2.0,1.75,1.25,317.8654,348.4248,387.8819,72400006.0
3,2.0,NaN,1.666667,0.0,3.00,4.00,3.0,2.0,2.00,1.50,372.5356,424.7139,349.9696,72400007.0
4,0.0,2.0,0.000000,NaN,2.25,3.50,2.0,2.0,2.00,2.00,485.6010,497.4628,494.6975,72400010.0
5,3.0,NaN,3.000000,0.0,3.00,4.00,3.0,3.0,2.00,1.75,432.8490,323.1210,395.8614,72400011.0
6,2.0,1.0,1.000000,NaN,1.00,4.00,3.0,1.0,2.00,1.75,452.9222,518.1177,497.2441,72400012.0
7,2.5,1.0,2.000000,NaN,1.50,4.00,2.0,2.0,2.00,1.75,554.2281,581.3172,614.3696,72400014.0
8,2.0,1.0,2.000000,NaN,3.00,4.00,2.0,2.0,2.00,1.50,452.2383,449.5834,476.7276,72400015.0
9,3.0,0.0,3.000000,1.0,2.00,0.75,0.0,3.0,0.25,0.00,413.6566,485.9650,450.0799,72400016.0


In [86]:
df_students_feat.set_index("student_id", inplace = True)

In [87]:
df_students_feat.head(3)

,teach_friend,teach_mean,emo_impl_teach,teach_intim,edu_impl_teach,edu_impl_par,par_relation_sat,teach_relation_sat,emo_imp_parents,par_freedom,MATH_mean,READ_mean,SCIE_mean
student_id,,,,,,,,,,,,,
72400001.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,471.9383,455.4310,493.6576
72400003.0,2.5,NaN,1.666667,NaN,1.25,NaN,3.0,2.0,2.00,2.00,494.8669,537.5307,522.2897
72400006.0,2.0,1.0,2.000000,NaN,1.00,4.0,2.0,2.0,1.75,1.25,317.8654,348.4248,387.8819


In [88]:
df_students_feat.head(20).T

student_id,72400001.0,72400003.0,72400006.0,72400007.0,72400010.0,72400011.0,72400012.0,72400014.0,72400015.0,72400016.0,72400017.0,72400018.0,72400019.0,72400020.0,72400021.0,72400022.0,72400023.0,72400024.0,72400026.0,72400028.0
teach_friend,NaN,2.500000,2.0000,2.000000,0.0000,3.0000,2.0000,2.5000,2.0000,3.0000,3.000000,0.0000,2.5000,1.000000,2.000000,3.0000,3.000000,0.0000,3.0000,1.0000
teach_mean,NaN,NaN,1.0000,NaN,2.0000,NaN,1.0000,1.0000,1.0000,0.0000,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000,NaN,0.0000,2.0000
emo_impl_teach,NaN,1.666667,2.0000,1.666667,0.0000,3.0000,1.0000,2.0000,2.0000,3.0000,2.333333,0.0000,2.0000,0.333333,1.666667,2.0000,1.333333,0.0000,2.0000,0.5000
teach_intim,NaN,NaN,NaN,0.000000,NaN,0.0000,NaN,NaN,NaN,1.0000,0.000000,NaN,0.0000,2.000000,NaN,1.0000,NaN,0.0000,0.0000,2.0000
edu_impl_teach,NaN,1.250000,1.0000,3.000000,2.2500,3.0000,1.0000,1.5000,3.0000,2.0000,NaN,2.0000,3.0000,1.500000,1.250000,1.2500,1.250000,NaN,2.7500,0.5000
edu_impl_par,NaN,NaN,4.0000,4.000000,3.5000,4.0000,4.0000,4.0000,4.0000,0.7500,NaN,2.0000,1.2500,2.666667,1.000000,NaN,NaN,NaN,2.5000,NaN
par_relation_sat,NaN,3.000000,2.0000,3.000000,2.0000,3.0000,3.0000,2.0000,2.0000,0.0000,2.000000,2.0000,2.0000,3.000000,1.000000,3.0000,0.000000,NaN,3.0000,2.0000
teach_relation_sat,NaN,2.000000,2.0000,2.000000,2.0000,3.0000,1.0000,2.0000,2.0000,3.0000,2.000000,2.0000,2.0000,0.000000,2.000000,3.0000,2.000000,NaN,2.0000,1.0000
emo_imp_parents,NaN,2.000000,1.7500,2.000000,2.0000,2.0000,2.0000,2.0000,2.0000,0.2500,2.000000,1.5000,NaN,2.000000,0.500000,2.0000,0.000000,NaN,2.0000,1.0000
par_freedom,NaN,2.000000,1.2500,1.500000,2.0000,1.7500,1.7500,1.7500,1.5000,0.0000,1.500000,2.0000,NaN,1.500000,1.000000,1.2500,1.000000,NaN,1.2500,1.5000


In [89]:
df_students_feat.shape

(30800, 13)

### Gestión de nulos

Vemos numero de nulos en las filas

In [90]:
# Count null values per row
null_counts = df_students_feat.isnull().sum(axis=1)
null_distribution = null_counts.value_counts().sort_index()

In [91]:
# Mostrar la distribución
print("Distribución de filas según número de valores nulos:")
print("Número de nulos | Número de filas")
print("-" * 35)
for n_nulls, n_rows in null_distribution.items():
    print(f"{n_nulls:^14} | {n_rows:^14}")

Distribución de filas según número de valores nulos:
Número de nulos | Número de filas
-----------------------------------
      0        |      6738     
      1        |     14483     
      2        |      5577     
      3        |      1080     
      4        |      557      
      5        |      656      
      6        |      451      
      7        |      205      
      8        |       51      
      9        |       21      
      10       |      981      


Vamos a eliminar las filas (estudiantes) con más de 4 valores nulos.

In [92]:
# eliminar filas con mas de 4 valores nulos
df_students_feat = df_students_feat[df_students_feat.isnull().sum(axis=1) <= 4]


In [98]:

# Seleccionar las variables para el clustering
features = df_students_feat.columns.tolist()

# Crear el imputador KNN
imputer = KNNImputer(
    n_neighbors=5,  # número de vecinos más cercanos
    weights='distance'  # da más peso a los vecinos más cercanos
)

# Imputar los valores
df_imputed = pd.DataFrame(
    imputer.fit_transform(df_students_feat[features]),
    columns = features
)

In [101]:
# verificamos

# Verificar la distribución de los datos antes y después
print("Distribución antes de la imputación:")
print(df_students_feat.describe())

print("\nDistribución después de la imputación:")
print(df_imputed.describe())

Distribución antes de la imputación:
       teach_friend    teach_mean  emo_impl_teach   teach_intim  \
count  25273.000000  17896.000000    28362.000000  17714.000000   
mean       2.132988      0.611869        1.811729      0.693802   
std        0.638394      0.692838        0.675713      0.739830   
min        0.000000      0.000000        0.000000      0.000000   
25%        2.000000      0.000000        1.500000      0.000000   
50%        2.000000      1.000000        2.000000      1.000000   
75%        2.500000      1.000000        2.250000      1.000000   
max        3.000000      3.000000        3.000000      3.000000   

       edu_impl_teach  edu_impl_par  par_relation_sat  teach_relation_sat  \
count    27826.000000  24745.000000      27983.000000        27988.000000   
mean         1.982352      2.583877          2.213808            1.920001   
std          0.857064      1.170075          0.772485            0.695397   
min          0.000000      0.000000          0.0000